In [163]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline

In [164]:
# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, encoding="utf8") as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            (Id, Text, Label) = parseReview(line)
            rawData.append((Id, Text, Label))
            preprocessedData.append((Id, preProcess(Text), Label))
        
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (_, Text, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
    for (_, Text, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))

In [165]:
# QUESTION 1

# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    ID = 0
    TEXT = 8
    LABEL = 1
    tuple = (reviewLine[ID], reviewLine[TEXT], reviewLine[LABEL])
    print("tuple created..." + reviewLine[ID])
    return tuple


In [166]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

# Input: a string of one review
def preProcess(text):
    # Should return a list of tokens
    return []

In [167]:
# QUESTION 2
featureDict = {} # A global dictionary of features

def toFeatureVector(tokens):
    # Should return a dictionary containing features as keys, and weights as values
    return {}

In [168]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)

In [169]:
# QUESTION 3

def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = len(dataset)/folds
    for i in range(0,len(dataset),foldSize):
        continue # Replace by code that trains and tests on the 10 folds of data in the dataset
    return cv_results

In [170]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: toFeatureVector(preProcess(t[1])), reviewSamples))

def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [171]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
reviewPath = 'amazon_reviews.txt'

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData(reviewPath) 
# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')
splitData(0.8)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
tuple created...DOC_ID
tuple created...1
tuple created...2
tuple created...3
tuple created...4
tuple created...5
tuple created...6
tuple created...7
tuple created...8
tuple created...9
tuple created...10
tuple created...11
tuple created...12
tuple created...13
tuple created...14
tuple created...15
tuple created...16
tuple created...17
tuple created...18
tuple created...19
tuple created...20
tuple created...21
tuple created...22
tuple created...23
tuple created...24
tuple created...25
tuple created...26
tuple created...27
tuple created...28
tuple created...29
tuple created...30
tuple created...31
tuple created...32
tuple created...33
tuple created...34
tuple created...35
tuple created...36
tuple created...37
tuple created...38
tuple created...39
tuple created...40
tuple created...41
tuple created...42
tuple created...43
tuple created...44
tuple created...45
tuple created...46
tuple created...47
tuple created...48
tuple crea

tuple created...1273
tuple created...1274
tuple created...1275
tuple created...1276
tuple created...1277
tuple created...1278
tuple created...1279
tuple created...1280
tuple created...1281
tuple created...1282
tuple created...1283
tuple created...1284
tuple created...1285
tuple created...1286
tuple created...1287
tuple created...1288
tuple created...1289
tuple created...1290
tuple created...1291
tuple created...1292
tuple created...1293
tuple created...1294
tuple created...1295
tuple created...1296
tuple created...1297
tuple created...1298
tuple created...1299
tuple created...1300
tuple created...1301
tuple created...1302
tuple created...1303
tuple created...1304
tuple created...1305
tuple created...1306
tuple created...1307
tuple created...1308
tuple created...1309
tuple created...1310
tuple created...1311
tuple created...1312
tuple created...1313
tuple created...1314
tuple created...1315
tuple created...1316
tuple created...1317
tuple created...1318
tuple created...1319
tuple created

tuple created...3431
tuple created...3432
tuple created...3433
tuple created...3434
tuple created...3435
tuple created...3436
tuple created...3437
tuple created...3438
tuple created...3439
tuple created...3440
tuple created...3441
tuple created...3442
tuple created...3443
tuple created...3444
tuple created...3445
tuple created...3446
tuple created...3447
tuple created...3448
tuple created...3449
tuple created...3450
tuple created...3451
tuple created...3452
tuple created...3453
tuple created...3454
tuple created...3455
tuple created...3456
tuple created...3457
tuple created...3458
tuple created...3459
tuple created...3460
tuple created...3461
tuple created...3462
tuple created...3463
tuple created...3464
tuple created...3465
tuple created...3466
tuple created...3467
tuple created...3468
tuple created...3469
tuple created...3470
tuple created...3471
tuple created...3472
tuple created...3473
tuple created...3474
tuple created...3475
tuple created...3476
tuple created...3477
tuple created

tuple created...5711
tuple created...5712
tuple created...5713
tuple created...5714
tuple created...5715
tuple created...5716
tuple created...5717
tuple created...5718
tuple created...5719
tuple created...5720
tuple created...5721
tuple created...5722
tuple created...5723
tuple created...5724
tuple created...5725
tuple created...5726
tuple created...5727
tuple created...5728
tuple created...5729
tuple created...5730
tuple created...5731
tuple created...5732
tuple created...5733
tuple created...5734
tuple created...5735
tuple created...5736
tuple created...5737
tuple created...5738
tuple created...5739
tuple created...5740
tuple created...5741
tuple created...5742
tuple created...5743
tuple created...5744
tuple created...5745
tuple created...5746
tuple created...5747
tuple created...5748
tuple created...5749
tuple created...5750
tuple created...5751
tuple created...5752
tuple created...5753
tuple created...5754
tuple created...5755
tuple created...5756
tuple created...5757
tuple created

tuple created...7582
tuple created...7583
tuple created...7584
tuple created...7585
tuple created...7586
tuple created...7587
tuple created...7588
tuple created...7589
tuple created...7590
tuple created...7591
tuple created...7592
tuple created...7593
tuple created...7594
tuple created...7595
tuple created...7596
tuple created...7597
tuple created...7598
tuple created...7599
tuple created...7600
tuple created...7601
tuple created...7602
tuple created...7603
tuple created...7604
tuple created...7605
tuple created...7606
tuple created...7607
tuple created...7608
tuple created...7609
tuple created...7610
tuple created...7611
tuple created...7612
tuple created...7613
tuple created...7614
tuple created...7615
tuple created...7616
tuple created...7617
tuple created...7618
tuple created...7619
tuple created...7620
tuple created...7621
tuple created...7622
tuple created...7623
tuple created...7624
tuple created...7625
tuple created...7626
tuple created...7627
tuple created...7628
tuple created

...

tuple created...11972
tuple created...11973
tuple created...11974
tuple created...11975
tuple created...11976
tuple created...11977
tuple created...11978
tuple created...11979
tuple created...11980
tuple created...11981
tuple created...11982
tuple created...11983
tuple created...11984
tuple created...11985
tuple created...11986
tuple created...11987
tuple created...11988
tuple created...11989
tuple created...11990
tuple created...11991
tuple created...11992
tuple created...11993
tuple created...11994
tuple created...11995
tuple created...11996
tuple created...11997
tuple created...11998
tuple created...11999
tuple created...12000
tuple created...12001
tuple created...12002
tuple created...12003
tuple created...12004
tuple created...12005
tuple created...12006
tuple created...12007
tuple created...12008
tuple created...12009
tuple created...12010
tuple created...12011
tuple created...12012
tuple created...12013
tuple created...12014
tuple created...12015
tuple created...12016
tuple crea

tuple created...14424
tuple created...14425
tuple created...14426
tuple created...14427
tuple created...14428
tuple created...14429
tuple created...14430
tuple created...14431
tuple created...14432
tuple created...14433
tuple created...14434
tuple created...14435
tuple created...14436
tuple created...14437
tuple created...14438
tuple created...14439
tuple created...14440
tuple created...14441
tuple created...14442
tuple created...14443
tuple created...14444
tuple created...14445
tuple created...14446
tuple created...14447
tuple created...14448
tuple created...14449
tuple created...14450
tuple created...14451
tuple created...14452
tuple created...14453
tuple created...14454
tuple created...14455
tuple created...14456
tuple created...14457
tuple created...14458
tuple created...14459
tuple created...14460
tuple created...14461
tuple created...14462
tuple created...14463
tuple created...14464
tuple created...14465
tuple created...14466
tuple created...14467
tuple created...14468
tuple crea

tuple created...16567
tuple created...16568
tuple created...16569
tuple created...16570
tuple created...16571
tuple created...16572
tuple created...16573
tuple created...16574
tuple created...16575
tuple created...16576
tuple created...16577
tuple created...16578
tuple created...16579
tuple created...16580
tuple created...16581
tuple created...16582
tuple created...16583
tuple created...16584
tuple created...16585
tuple created...16586
tuple created...16587
tuple created...16588
tuple created...16589
tuple created...16590
tuple created...16591
tuple created...16592
tuple created...16593
tuple created...16594
tuple created...16595
tuple created...16596
tuple created...16597
tuple created...16598
tuple created...16599
tuple created...16600
tuple created...16601
tuple created...16602
tuple created...16603
tuple created...16604
tuple created...16605
tuple created...16606
tuple created...16607
tuple created...16608
tuple created...16609
tuple created...16610
tuple created...16611
tuple crea

tuple created...18363
tuple created...18364
tuple created...18365
tuple created...18366
tuple created...18367
tuple created...18368
tuple created...18369
tuple created...18370
tuple created...18371
tuple created...18372
tuple created...18373
tuple created...18374
tuple created...18375
tuple created...18376
tuple created...18377
tuple created...18378
tuple created...18379
tuple created...18380
tuple created...18381
tuple created...18382
tuple created...18383
tuple created...18384
tuple created...18385
tuple created...18386
tuple created...18387
tuple created...18388
tuple created...18389
tuple created...18390
tuple created...18391
tuple created...18392
tuple created...18393
tuple created...18394
tuple created...18395
tuple created...18396
tuple created...18397
tuple created...18398
tuple created...18399
tuple created...18400
tuple created...18401
tuple created...18402
tuple created...18403
tuple created...18404
tuple created...18405
tuple created...18406
tuple created...18407
tuple crea

tuple created...20371
tuple created...20372
tuple created...20373
tuple created...20374
tuple created...20375
tuple created...20376
tuple created...20377
tuple created...20378
tuple created...20379
tuple created...20380
tuple created...20381
tuple created...20382
tuple created...20383
tuple created...20384
tuple created...20385
tuple created...20386
tuple created...20387
tuple created...20388
tuple created...20389
tuple created...20390
tuple created...20391
tuple created...20392
tuple created...20393
tuple created...20394
tuple created...20395
tuple created...20396
tuple created...20397
tuple created...20398
tuple created...20399
tuple created...20400
tuple created...20401
tuple created...20402
tuple created...20403
tuple created...20404
tuple created...20405
tuple created...20406
tuple created...20407
tuple created...20408
tuple created...20409
tuple created...20410
tuple created...20411
tuple created...20412
tuple created...20413
tuple created...20414
tuple created...20415
tuple crea